In [1]:
from cloud_spider import NWSpider

In [2]:
url = 'https://www.networkworld.com/cloud-computing/'
spider = NWSpider(url)
spider.extract()

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime as dt
from sqlalchemy import create_engine
import os
from urllib.request import Request, urlopen

In [16]:
class CloudSpider():
    def __init__(self):
        conn_url = os.environ.get('POSTGRES_CONNECTION_STRING')
        self.engine = create_engine(f'{conn_url}cloudnewsdb')
        self.data = []
    
    @staticmethod
    def get_soup(url):
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = urlopen(req).read()
        soup = BeautifulSoup(page, 'lxml')
        return soup

    def extract_nw(self):
        soup = self.get_soup('https://www.networkworld.com/cloud-computing/')
        for a in soup.find_all('a', attrs={'class': 'card'}):
            date = a.find_next('div', attrs={'class': 'card__info--light'}).find_next('span').text
            if 'dec' in date.lower():
                self.data.append({
                    'title': a.find_next('h4').text,
                    'url': a['href'],
                    'date': date,
                    'source': 'Network World',
                    'query_date': dt.now()
                })
                
    def extract_cd(self):
        soup = self.get_soup('https://www.ciodive.com/topic/cloud/')
        for li in soup.select('li.row.feed__item'):
            a = li.find_next('h3', attrs={'class': 'feed__title'}).find_next('a')
            url = 'https://www.ciodive.com' + a['href']
            title = a.text.strip()
            try:
                date = li.find_all('span', attrs={'class':'secondary-label'})[1].text
            except:
                pass
            self.data.append({
            'title':title,
            'url': url,
            'date': date,
            'source': 'Ciodive',
            'query_date': dt.now()
            })
            
    def extract_cc(self):
        soup = self.get_soup('https://www.cloudcomputing-news.net/categories/cloud-computing/')
        for article in soup.find_all('article', attrs={'class':'archive-post'}):
            a =  article.find_next('header').find_next('a')
            byline = article.find_next('div', attrs={'class':'byline'})
            self.data.append({
            'title':a.text.strip(),
            'url': a['href'],
            'date': byline.text.split('|')[0].strip(),
            'source': 'Cloudtech',
            'query_date': dt.now()
            })
            
    def process_extraction(self):
        self.extract_nw()
        self.extract_cd()
        self.extract_cc()
            
    def get_df(self):
        return pd.DataFrame(self.data)

    def update_sql(self):            
        df = pd.DataFrame(self.data)
        df.to_sql('raw', self.engine, if_exists='replace')

In [17]:
spider = CloudSpider()
spider.process_extraction()
data = spider.get_df()

In [18]:
data

,title,url,date,source,query_date
0,Arista: AI and cloud networking portend bright...,https://www.networkworld.com/article/1267632/a...,"Dec 19, 2023",Network World,2024-01-12 21:11:33.128256
1,Oxide puts the 'cloud' back in on-prem private...,https://www.networkworld.com/article/1250120/s...,"Dec 12, 2023",Network World,2024-01-12 21:11:33.128256
2,"IBM cloud service aims to deliver secure, mult...",https://www.networkworld.com/article/1251939/i...,"Dec 07, 2023",Network World,2024-01-12 21:11:33.128256
3,"EU approves $1.3B in aid for cloud, edge compu...",https://www.networkworld.com/article/1251281/e...,"Dec 05, 2023",Network World,2024-01-12 21:11:33.128256
4,"SAP adds board-level cloud migration unit, res...",/news/sap-board-add-cloud-unit-saas-enterprise...,"Jan. 11, 2024",Ciodive,2024-01-12 21:11:34.095243
5,"Nvidia unveils laptop AI chip, developer toolkit",/news/nvidia-laptop-pc-gpu-generative-ai-proce...,"Jan. 10, 2024",Ciodive,2024-01-12 21:11:34.095243
6,Hybrid cloud,/trendline/hybrid-cloud/391/,"Jan. 10, 2024",Ciodive,2024-01-12 21:11:34.095243
7,"CompTIA bolsters training portfolio, adds AI f...",/news/comptia-ai-cloud-data-cyber-training-cer...,"Jan. 3, 2024",Ciodive,2024-01-12 21:11:34.095243
8,Enterprises plan to boost reliance on distribu...,/news/distributed-cloud-edge-workloads-analyti...,"Jan. 2, 2024",Ciodive,2024-01-12 21:11:34.095243
9,Industry clouds set for slow start in the new ...,/news/industry-cloud-solutions-spread-stalls-F...,"Dec. 21, 2023",Ciodive,2024-01-12 21:11:34.095243


In [8]:
dt.strftime(dt.today(), format='%b').lower()

'jan'

In [3]:
url = 'https://www.ciodive.com/topic/cloud/'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [4]:
data = []
for li in soup.select('li.row.feed__item'):
    a = li.find_next('h3', attrs={'class': 'feed__title'}).find_next('a')
    url = a['href']
    title = a.text.strip()
    try:
        date = li.find_all('span', attrs={'class':'secondary-label'})[1].text
    except:
        pass
    data.append({
    'title':title,
    'url': url,
    'date': date,
    'source': 'Ciodive',
    'query_date': dt.now()
    })
data

[{'title': 'SAP adds board-level cloud migration unit, reshuffles executive roles',
  'url': '/news/sap-board-add-cloud-unit-saas-enterprise-software-shift/704273/',
  'date': 'Jan. 11, 2024',
  'source': 'Ciodive',
  'query_date': datetime.datetime(2024, 1, 12, 21, 3, 49, 546835)},
 {'title': 'Nvidia unveils laptop AI chip, developer toolkit',
  'url': '/news/nvidia-laptop-pc-gpu-generative-ai-processor/704227/',
  'date': 'Jan. 10, 2024',
  'source': 'Ciodive',
  'query_date': datetime.datetime(2024, 1, 12, 21, 3, 49, 546835)},
 {'title': 'Hybrid cloud',
  'url': '/trendline/hybrid-cloud/391/',
  'date': 'Jan. 10, 2024',
  'source': 'Ciodive',
  'query_date': datetime.datetime(2024, 1, 12, 21, 3, 49, 547836)},
 {'title': 'CompTIA bolsters training portfolio, adds AI fundamentals and AWS pro certs',
  'url': '/news/comptia-ai-cloud-data-cyber-training-certifications/703594/',
  'date': 'Jan. 3, 2024',
  'source': 'Ciodive',
  'query_date': datetime.datetime(2024, 1, 12, 21, 3, 49, 547

In [5]:
url = 'https://www.cloudcomputing-news.net/categories/cloud-computing/'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(req).read()
soup = BeautifulSoup(page, 'lxml')

In [8]:
data = []
for article in soup.find_all('article', attrs={'class':'archive-post'}):
    a =  article.find_next('header').find_next('a')
    byline = article.find_next('div', attrs={'class':'byline'})
    data.append({
    'title':a.text.strip(),
    'url': a['href'],
    'date': byline.text.split('|')[0].strip(),
    'source': 'Cloudtech',
    'query_date': dt.now()
    })
data

[{'title': 'Delinea acquires Authomize to strengthen access management',
  'url': 'https://www.cloudcomputing-news.net/news/2024/jan/10/delinea-acquires-authomize-to-strengthen-access-management/',
  'date': '10 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 21, 7, 29, 638951)},
 {'title': 'Zayo Group’s ‘near-net’ expansion to bring 100G connectivity to Western Europe',
  'url': 'https://www.cloudcomputing-news.net/news/2024/jan/09/zayo-groups-near-net-expansion-to-bring-100g-connectivity-to-western-europe/',
  'date': '9 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 21, 7, 29, 638951)},
 {'title': 'Lenovo launches hybrid cloud platforms and services to accelerate AI',
  'url': 'https://www.cloudcomputing-news.net/news/2024/jan/02/lenovo-launches-hybrid-cloud-platforms-and-services-to-accelerate-ai/',
  'date': '2 January 2024',
  'source': 'Cloudtech',
  'query_date': datetime.datetime(2024, 1, 12, 21, 7,